# Animated Lorenz Attractor

In [1]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt, glob
import IPython.display as IPdisplay, matplotlib.font_manager as fm
from scipy.integrate import odeint
from mpl_toolkits.mplot3d.axes3d import Axes3D
from PIL import Image as PIL_Image
from images2gif import writeGif

In [2]:
# define the fonts to use for plots
family = 'Myriad Pro'
title_font = fm.FontProperties(family=family, style='normal', size=20, weight='normal', stretch='normal')

In [3]:
# define the lorenz system
def lorenz_system(current_state, t):
    x, y, z = current_state
    dx_dt = sigma * (y - x)
    dy_dt = x * (rho - z) - y
    dz_dt = x * y - beta * z
    return [dx_dt, dy_dt, dz_dt]

In [4]:
# plot the system in 3 dimensions
def plot_lorenz(xyz, n):
    fig = plt.figure(figsize=(12, 9))
    ax = fig.gca(projection='3d')
    ax.xaxis.set_pane_color((1,1,1,1))
    ax.yaxis.set_pane_color((1,1,1,1))
    ax.zaxis.set_pane_color((1,1,1,1))
    x = xyz[:, 0]
    y = xyz[:, 1]
    z = xyz[:, 2]
    ax.plot(x, y, z, color='g', alpha=0.7, linewidth=0.6)
    ax.set_xlim((-30,30))
    ax.set_ylim((-30,30))
    ax.set_zlim((0,50))
    ax.set_title('Lorenz Attractor', fontproperties=title_font)
    plt.savefig('images/{}/{:03d}.png'.format(save_folder, n), dpi=60, bbox_inches='tight', pad_inches=0.1)
    plt.close()

In [5]:
# return a list in iteratively larger chunks
def get_chunks(l, size):
    size = max(1, size)
    chunks = []
    for i in range(1, len(l)+1, size):
        chunks.append(l[0:i])    
    return chunks

In [6]:
save_folder = 'lorenz-animate'

# define the initial system state (aka x, y, z positions in space)
initial_state = [0.1, 0, 0]

# define the system parameters sigma, rho, and beta
sigma = 10.
rho   = 28.
beta  = 8./3.

# define the time points to solve for, evenly spaced between the start and end times
start_time = 1
end_time = 60
interval = 100
end_pause = 20
time_points = np.linspace(start_time, end_time, end_time * interval)

In [7]:
# get incrementally larger chunks of the time points, to reveal the attractor one frame at a time
chunks = get_chunks(time_points, size=20)

# append the full sequence at the end to make it display the whole thing for end_pause number of frames
for _ in range(end_pause):
    chunks.append(time_points)

In [8]:
# get the points to plot, one chunk of time steps at a time, by integrating the system of equations
points = []
for chunk in chunks:
    xyz = odeint(lorenz_system, initial_state, chunk)
    points.append(xyz)

In [9]:
# plot each set of points, one at a time, saving each plot
for xyz, n in zip(points, range(len(points))):
    plot_lorenz(xyz, n)

In [10]:
# create an animated gif of all the plots then display it inline
images = [PIL_Image.open(image) for image in glob.glob('images/{}/*.png'.format(save_folder))]
gif_filename = 'images/{}/animated-lorenz-attractor.gif'.format(save_folder)
writeGif(gif_filename, images, duration=0.05)
IPdisplay.Image(url=gif_filename)